In [2]:
import numpy as np
import pandas as pd
#from data_processing_helper import *
import warnings
warnings.filterwarnings('ignore')
from statistics import multimode
from statistics import mode
import itertools
import os
import tqdm
import re

In [3]:
def cleanup(filepath):
    #print(filepath)
    # import data and transpose
    df_raw = pd.read_excel(filepath,header=None).T

    # modify the header
    new_header = df_raw.iloc[0]   #grab the first row for the header
    df = df_raw[1:]               #take the data except the header row
    df.columns = new_header 
    df.reset_index(drop=True, inplace=True)

    # add new column for visualization 
    df['Start Datetime'] = pd.to_datetime(df['Start Date'] + ' ' + df['Start Time'])
    df['End Datetime'] = pd.to_datetime(df['End Date'] + ' ' + df['End Time'])
    df['Timeout'] = (df['Active Lever Presses'] - df['Reward']).astype('int32')

    # drop unnecessary columns
    df.drop(['Start Date','Start Time','End Date','End Time'], axis=1, inplace=True)

    # change data types
    cols = df.columns.tolist()
    for col in cols[:-1]:
        name = col.lower()
        if ('active' in name) or ('reward' in name) or ('timeout' in name):
            df[col] = df[col].astype('int32')
            df[col] = df[col].replace(0,np.nan)
        else:
            pass
        
    # drop columns which all values are Nan
    df.dropna(how='all', axis=1, inplace=True)

    # reorder the columns
    new_columns = df.columns.tolist()
    new_columns.insert(0, new_columns.pop(new_columns.index('Subject')))
    new_columns.insert(1, new_columns.pop(new_columns.index('Start Datetime')))
    new_columns.insert(2, new_columns.pop(new_columns.index('End Datetime')))

    #
    #idx = new_columns.index('Reward') + 1
    #new_columns.insert(idx, new_columns.pop(new_columns.index('Timeout')))

    # fill the nan with 0
    df = df[new_columns]
    df.fillna(0,inplace=True)
    #print('CLEANING COMPLETED')
    # output the result dataframe
    return df

In [4]:
# helper functions
def get_mode(lst):
    if len(lst) == 0:
        return np.NaN
    return mode(lst)


def get_bursts(lst):
    interval = 120
    allBursts = []

    i = 0
    while i < len(lst):
        oneBurst = []
        limit = lst[i] + interval
        j = i
        while j < len(lst) and lst[j] <= limit:
            oneBurst.append(lst[j])
            j += 1
        allBursts.append(oneBurst)
        i = j
        
    return allBursts


def get_mean_num_rewards(lst):
    bursts = [i for i in lst if len(i) > 1]
    total_rewards = len(list(itertools.chain.from_iterable(bursts)))
    total_bursts = len(bursts)
    
    if total_bursts == 0: 
        return 0
    
    return round(total_rewards / total_bursts,2)


def get_burst_rewards_pct(lst):
    numRewards = len(list(itertools.chain.from_iterable(lst)))
    if numRewards == 0:
        return np.NaN
    bursts = [i for i in lst if len(i) > 1]
    numBurstRewards = len(list(itertools.chain.from_iterable(bursts)))
    return round(numBurstRewards/numRewards,2) * 100 


def get_max_burst(lst):
    bursts = [i for i in lst if len(i) > 1]
    if len(bursts) == 0:
        return 0
    return max([len(i) for i in bursts])

In [5]:
def calculations_single(df):
    # combine all reward timestamp into a column of lists
    filtered_cols = ['Subject'] + [col for col in df.columns if 'Reward ' in col]
    df_reward = df[filtered_cols].sort_values('Subject').reset_index().drop('index',axis=1)
    df_reward['allRewards'] = df_reward.iloc[:,1:].values.tolist()

    # get the filtered df
    dff = df_reward[['Subject', 'allRewards']]

    # retrieve the inter-reward intervals, filtering out negatives
    dff['Intervals'] = dff['allRewards'].apply(lambda lst:[j-i for i, j in zip(lst[:-1], lst[1:])])
    dff['cleanedIntervals'] = dff['Intervals'].apply(lambda lst: [val for val in lst if val > 0])

    # calculate needed traits related to the inter-reward intervals
    dff['meanInterval'] = dff['cleanedIntervals'].apply(lambda x: round(np.mean(x), 2))
    dff['stdInterval'] = dff['cleanedIntervals'].apply(lambda x: round(np.std(x), 2))
    dff['modeInterval'] = dff['cleanedIntervals'].apply(get_mode)

    # filtering the rewards (zero values)
    dff['cleanedRewards'] = dff['allRewards'].apply(lambda lst: [val for val in lst if val > 0])
    dff['totalRewards'] = dff['cleanedRewards'].apply(lambda x: len(x))

    # retrieve the "bursts" (cluster of rewards happened within 2 mins) from rewards 
    dff['rawBurst'] = dff['cleanedRewards'].apply(get_bursts)
    dff['numBurst'] = dff['rawBurst'].apply(lambda x: len([i for i in x if len(i)>1]))

    # get the mean number of rewards across all the bursts
    dff['meanNumRewards'] = dff['rawBurst'].apply(get_mean_num_rewards)

    # get the percentage of rewards that fall in burst out of all the rewards
    dff['pctRewards'] = dff['rawBurst'].apply(get_burst_rewards_pct)

    # get the maximum number of rewards contain in a single burst in one session
    dff['maxBurst'] = dff['rawBurst'].apply(get_max_burst)

    # select needed columns for output df
    output_cols = ['Subject', 'meanInterval', 'stdInterval', 'modeInterval','meanNumRewards', 'numBurst', 'maxBurst', 'pctRewards']
    dff_out = dff[output_cols]
    
    return dff_out

## SHA 01-04

In [6]:
files = [i for i in sorted(os.listdir('./SHA')) if int(i[-13]) in [1,2,3,4]]
files

['BSB273BC04HSOXYSHA01_output.xlsx',
 'BSB273BC04HSOXYSHA02_output.xlsx',
 'BSB273BC04HSOXYSHA03_output.xlsx',
 'BSB273BC04HSOXYSHA04_output.xlsx',
 'BSB273BC05HSOXYSHA01_output.xlsx',
 'BSB273BC05HSOXYSHA02_output.xlsx',
 'BSB273BC05HSOXYSHA03_output.xlsx',
 'BSB273BC05HSOXYSHA04_output.xlsx',
 'BSB273BC06HSOXYSHA01_output.xlsx',
 'BSB273BC06HSOXYSHA02_output.xlsx',
 'BSB273BC06HSOXYSHA03_output.xlsx',
 'BSB273BC06HSOXYSHA04_output.xlsx',
 'BSB273BC07HSOXYSHA01_output.xlsx',
 'BSB273BC07HSOXYSHA02_output.xlsx',
 'BSB273BC07HSOXYSHA03_output.xlsx',
 'BSB273BC07HSOXYSHA04_output.xlsx',
 'BSB273BC09HSOXYSHA01_output.xlsx',
 'BSB273BC09HSOXYSHA02_output.xlsx',
 'BSB273BC09HSOXYSHA03_output.xlsx',
 'BSB273BC09HSOXYSHA04_output.xlsx',
 'BSB273BC10HSOXYSHA01_output.xlsx',
 'BSB273BC10HSOXYSHA02_output.xlsx',
 'BSB273BC10HSOXYSHA03_output.xlsx',
 'BSB273BC10HSOXYSHA04_output.xlsx',
 'BSB273BC11HSOXYSHA01_output.xlsx',
 'BSB273BC11HSOXYSHA02_output.xlsx',
 'BSB273BC11HSOXYSHA03_output.xlsx',
 

In [7]:
calcs = []
for f in files:
    print(f)
    filepath = './SHA' + '//' + f
    df_cleaned = cleanup(filepath)
    calc = calculations_single(df_cleaned)
    calcs.append(calc)

BSB273BC04HSOXYSHA01_output.xlsx
BSB273BC04HSOXYSHA02_output.xlsx
BSB273BC04HSOXYSHA03_output.xlsx
BSB273BC04HSOXYSHA04_output.xlsx
BSB273BC05HSOXYSHA01_output.xlsx
BSB273BC05HSOXYSHA02_output.xlsx
BSB273BC05HSOXYSHA03_output.xlsx
BSB273BC05HSOXYSHA04_output.xlsx
BSB273BC06HSOXYSHA01_output.xlsx
BSB273BC06HSOXYSHA02_output.xlsx
BSB273BC06HSOXYSHA03_output.xlsx
BSB273BC06HSOXYSHA04_output.xlsx
BSB273BC07HSOXYSHA01_output.xlsx
BSB273BC07HSOXYSHA02_output.xlsx
BSB273BC07HSOXYSHA03_output.xlsx
BSB273BC07HSOXYSHA04_output.xlsx
BSB273BC09HSOXYSHA01_output.xlsx
BSB273BC09HSOXYSHA02_output.xlsx
BSB273BC09HSOXYSHA03_output.xlsx
BSB273BC09HSOXYSHA04_output.xlsx
BSB273BC10HSOXYSHA01_output.xlsx
BSB273BC10HSOXYSHA02_output.xlsx
BSB273BC10HSOXYSHA03_output.xlsx
BSB273BC10HSOXYSHA04_output.xlsx
BSB273BC11HSOXYSHA01_output.xlsx
BSB273BC11HSOXYSHA02_output.xlsx
BSB273BC11HSOXYSHA03_output.xlsx
BSB273BC11HSOXYSHA04_output.xlsx
BSB273BC12HSOXYSHA01_output.xlsx
BSB273BC12HSOXYSHA02_output.xlsx
BSB273BC12

In [19]:
df_rawcalcs = pd.concat(calcs, ignore_index=True)
df_rawcalcs.head()

,Subject,meanInterval,stdInterval,modeInterval,meanNumRewards,numBurst,maxBurst,pctRewards
0,F401,755.86,520.27,1356.0,2.0,2,2,50.0
1,F402,1730.25,2211.53,1327.0,2.0,1,2,40.0
2,F403,NaN,NaN,NaN,0.0,0,0,0.0
3,F404,519.00,354.00,165.0,0.0,0,0,0.0
4,F405,591.50,396.56,27.0,2.0,1,2,22.0


In [22]:
df_rawcalcs.Subject.value_counts()

0        13
F401      4
F110      4
M1380     4
F101      4
         ..
M878      1
M879      1
M880      1
F451      1
M872      1
Name: Subject, Length: 806, dtype: int64

In [23]:
rows = []
subs = [i for i in df_rawcalcs.Subject.unique() if i != '0']
for s in subs:
    temp = df_rawcalcs[df_rawcalcs.Subject == s]
    temp = temp.reset_index(drop=True)
    temp_row = temp.iloc[:,1:].unstack().to_frame().sort_index(level=1).T
    temp_row.insert(loc=0, column='Subject', value=temp.Subject.unique())
    rows.append(temp_row)

In [157]:
new_colname = ['Subject','maxBurst_SHA01','maxBurst_SHA02','maxBurst_SHA03','maxBurst_SHA04',
               'meanInterval_SHA01','meanInterval_SHA02','meanInterval_SHA03','meanInterval_SHA04',
               'meanNumRewards_SHA01','meanNumRewards_SHA02','meanNumRewards_SHA03','meanNumRewards_SHA04',
               'modeInterval_SHA01','modeInterval_SHA02','modeInterval_SHA03','modeInterval_SHA04',
               'numBurst_SHA01','numBurst_SHA02','numBurst_SHA03','numBurst_SHA04',
               'pctRewards_SHA01','pctRewards_SHA02','pctRewards_SHA03','pctRewards_SHA04',
               'stdInterval_SHA01','stdInterval_SHA02','stdInterval_SHA03','stdInterval_SHA04']

In [164]:
out = pd.concat(rows, ignore_index=True)
out.columns = new_colname
#out.to_csv('SHA_calculations.csv')
out.sort_values('Subject')

,Subject,maxBurst_SHA01,maxBurst_SHA02,maxBurst_SHA03,maxBurst_SHA04,meanInterval_SHA01,meanInterval_SHA02,meanInterval_SHA03,meanInterval_SHA04,meanNumRewards_SHA01,...,numBurst_SHA03,numBurst_SHA04,pctRewards_SHA01,pctRewards_SHA02,pctRewards_SHA03,pctRewards_SHA04,stdInterval_SHA01,stdInterval_SHA02,stdInterval_SHA03,stdInterval_SHA04
80,F1001,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,F1002,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,F1003,0.0,0.0,0.0,0.0,NaN,NaN,5910.00,3765.00,0.0,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.00,0.00
83,F1004,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,F1005,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,M966,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
482,M967,2.0,3.0,3.0,3.0,1269.8,425.4,839.57,494.88,2.0,...,2.0,2.0,67.0,83.0,62.0,56.0,1462.94,488.69,778.24,589.16
483,M968,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,M969,0.0,2.0,2.0,2.0,623.0,445.5,762.29,868.00,0.0,...,1.0,1.0,0.0,29.0,25.0,67.0,397.00,206.44,386.95,831.00


In [161]:
out.Subject.nunique()

805

## LGA 01-14

In [170]:
files = [i for i in sorted(os.listdir('./LGA01-14')) if 'LGA' in i]
files

['BSB273BC04HSOXYLGA01_output.xlsx',
 'BSB273BC04HSOXYLGA02_output.xlsx',
 'BSB273BC04HSOXYLGA03_output.xlsx',
 'BSB273BC04HSOXYLGA04_output.xlsx',
 'BSB273BC04HSOXYLGA05_output.xlsx',
 'BSB273BC04HSOXYLGA06_output.xlsx',
 'BSB273BC04HSOXYLGA07_output.xlsx',
 'BSB273BC04HSOXYLGA08_output.xlsx',
 'BSB273BC04HSOXYLGA09_output.xlsx',
 'BSB273BC04HSOXYLGA10_output.xlsx',
 'BSB273BC04HSOXYLGA11_output.xlsx',
 'BSB273BC04HSOXYLGA12_output.xlsx',
 'BSB273BC04HSOXYLGA13_output.xlsx',
 'BSB273BC04HSOXYLGA14_output.xlsx',
 'BSB273BC05HSOXYLGA01_output.xlsx',
 'BSB273BC05HSOXYLGA02_output.xlsx',
 'BSB273BC05HSOXYLGA03_output.xlsx',
 'BSB273BC05HSOXYLGA04_output.xlsx',
 'BSB273BC05HSOXYLGA05_output.xlsx',
 'BSB273BC05HSOXYLGA06_output.xlsx',
 'BSB273BC05HSOXYLGA07_output.xlsx',
 'BSB273BC05HSOXYLGA08_output.xlsx',
 'BSB273BC05HSOXYLGA09_output.xlsx',
 'BSB273BC05HSOXYLGA10_output.xlsx',
 'BSB273BC05HSOXYLGA11_output.xlsx',
 'BSB273BC05HSOXYLGA12_output.xlsx',
 'BSB273BC05HSOXYLGA13_output.xlsx',
 

In [171]:
len(files)

751

In [172]:
from collections import Counter
Counter([i[:10] for i in files])

Counter({'BSB273BC04': 14,
         'BSB273BC05': 14,
         'BSB273BC06': 14,
         'BSB273BC07': 14,
         'BSB273BC09': 14,
         'BSB273BC10': 14,
         'BSB273BC11': 14,
         'BSB273BC12': 14,
         'BSB273BC13': 14,
         'BSB273CC04': 14,
         'BSB273CC05': 14,
         'BSB273CC06': 14,
         'BSB273CC07': 14,
         'BSB273CC09': 14,
         'BSB273CC10': 14,
         'BSB273CC11': 14,
         'BSB273CC12': 14,
         'BSB273CC13': 13,
         'BSB273DC04': 14,
         'BSB273DC05': 14,
         'BSB273DC06': 14,
         'BSB273DC07': 14,
         'BSB273DC09': 14,
         'BSB273DC10': 14,
         'BSB273DC11': 14,
         'BSB273DC12': 14,
         'BSB273DC13': 14,
         'BSB273EC04': 14,
         'BSB273EC05': 14,
         'BSB273EC06': 14,
         'BSB273EC07': 14,
         'BSB273EC09': 14,
         'BSB273EC10': 14,
         'BSB273EC11': 14,
         'BSB273EC12': 14,
         'BSB273EC13': 14,
         'MED1110C01': 13,
 

In [173]:
len(Counter([i[:10] for i in files]))

54

In [174]:
cnt = Counter([i[:10] for i in files])
for k,v in enumerate(cnt):
    if cnt[v] != 14:
        print(v, cnt[v])

BSB273CC13 13
MED1110C01 13
MED1113C01 13
MTF134CC16 12


### Missing files (LGA)

- BSB273CC13: 13
- MED1110C01: 06
- MED1113C01: 06
- MED134CC16: 01, 11

### New cleanup function

In [178]:
def special_calculations_single(df):
    # combine all reward timestamp into a column of lists
    filtered_cols = ['Subject', 'Filename'] + [col for col in df.columns if 'Reward ' in col]
    df_reward = df[filtered_cols].sort_values('Subject').reset_index().drop('index',axis=1)
    #df_reward['Filename'] = df_reward['Filename'].apply(lambda x: re.findall(r'(LGA[0-9]+)',x.upper())[0])
    df_reward['allRewards'] = df_reward.iloc[:,2:].values.tolist()

    # get the filtered df
    # dff = df_reward[['Subject', 'Filename', 'allRewards']]
    dff = df_reward[['Subject', 'allRewards']]

    # retrieve the inter-reward intervals, filtering out negatives
    dff['Intervals'] = dff['allRewards'].apply(lambda lst:[j-i for i, j in zip(lst[:-1], lst[1:])])
    dff['cleanedIntervals'] = dff['Intervals'].apply(lambda lst: [val for val in lst if val > 0])

    # calculate needed traits related to the inter-reward intervals
    dff['meanInterval'] = dff['cleanedIntervals'].apply(lambda x: round(np.mean(x), 2))
    dff['stdInterval'] = dff['cleanedIntervals'].apply(lambda x: round(np.std(x), 2))
    dff['modeInterval'] = dff['cleanedIntervals'].apply(get_mode)

    # filtering the rewards (zero values)
    dff['cleanedRewards'] = dff['allRewards'].apply(lambda lst: [val for val in lst if val > 0])
    dff['totalRewards'] = dff['cleanedRewards'].apply(lambda x: len(x))

    # retrieve the "bursts" (cluster of rewards happened within 2 mins) from rewards 
    dff['rawBurst'] = dff['cleanedRewards'].apply(get_bursts)
    dff['numBurst'] = dff['rawBurst'].apply(lambda x: len([i for i in x if len(i)>1]))

    # get the mean number of rewards across all the bursts
    dff['meanNumRewards'] = dff['rawBurst'].apply(get_mean_num_rewards)

    # get the percentage of rewards that fall in burst out of all the rewards
    dff['pctRewards'] = dff['rawBurst'].apply(get_burst_rewards_pct)

    # get the maximum number of rewards contain in a single burst in one session
    dff['maxBurst'] = dff['rawBurst'].apply(get_max_burst)

    # select needed columns for output df
    # output_cols = ['Subject', 'Filename', 'meanInterval', 'stdInterval', 'modeInterval',
    #               'meanNumRewards', 'numBurst', 'maxBurst', 'pctRewards']
    output_cols = ['Subject', 'meanInterval', 'stdInterval', 'modeInterval',
                   'meanNumRewards', 'numBurst', 'maxBurst', 'pctRewards']
    dff_out = dff[output_cols]
    
    return dff_out

In [197]:
filepath = './LGA01-14' + '//' + 'MTF134DC16HSOXYLGA12_output.xlsx'
re.findall(r'(LGA[0-9]+)',filepath)

['LGA01', 'LGA12']

In [192]:
filepath = './LGA01-14' + '//' + 'MTF134DC16HSOXYLGA14_output.xlsx'
file_id = re.findall(r'(LGA[0-9]+)',filepath.upper())[1]
df_cleaned = cleanup(filepath)
calc = special_calculations_single(df_cleaned)
calc['file_id'] = [file_id] * len(calc)
calc

,Subject,meanInterval,stdInterval,modeInterval,meanNumRewards,numBurst,maxBurst,pctRewards,file_id
0,M1653,656.00,926.77,32.0,2.33,18,3,68.0,LGA01
1,M1654,1883.59,4222.71,135.0,2.00,5,2,43.0,LGA01
2,M1655,NaN,NaN,NaN,0.00,0,0,0.0,LGA01
3,M1656,8480.00,4358.00,12838.0,0.00,0,0,0.0,LGA01
4,M1657,512.00,1081.54,25.0,2.22,18,3,63.0,LGA01
5,M1658,1355.97,1886.21,32.0,2.00,4,2,26.0,LGA01
6,M1659,680.27,1286.13,235.0,2.15,13,3,49.0,LGA01
7,M1660,291.10,516.23,22.0,3.54,35,6,85.0,LGA01
8,M1661,782.72,325.94,42.0,2.00,2,2,10.0,LGA01
9,M1662,6000.80,5566.67,9898.0,2.00,2,2,67.0,LGA01


In [199]:
calcs = []
for f in files:
    print(f)
    filepath = './LGA01-14' + '//' + f
    file_id = re.findall(r'(LGA[0-9]+)',f)[0]
    #print(file_id)
    df_cleaned = cleanup(filepath)
    calc = special_calculations_single(df_cleaned)
    calc['file_id'] = [file_id] * len(calc)
    calcs.append(calc)

BSB273BC04HSOXYLGA01_output.xlsx
BSB273BC04HSOXYLGA02_output.xlsx
BSB273BC04HSOXYLGA03_output.xlsx
BSB273BC04HSOXYLGA04_output.xlsx
BSB273BC04HSOXYLGA05_output.xlsx
BSB273BC04HSOXYLGA06_output.xlsx
BSB273BC04HSOXYLGA07_output.xlsx
BSB273BC04HSOXYLGA08_output.xlsx
BSB273BC04HSOXYLGA09_output.xlsx
BSB273BC04HSOXYLGA10_output.xlsx
BSB273BC04HSOXYLGA11_output.xlsx
BSB273BC04HSOXYLGA12_output.xlsx
BSB273BC04HSOXYLGA13_output.xlsx
BSB273BC04HSOXYLGA14_output.xlsx
BSB273BC05HSOXYLGA01_output.xlsx
BSB273BC05HSOXYLGA02_output.xlsx
BSB273BC05HSOXYLGA03_output.xlsx
BSB273BC05HSOXYLGA04_output.xlsx
BSB273BC05HSOXYLGA05_output.xlsx
BSB273BC05HSOXYLGA06_output.xlsx
BSB273BC05HSOXYLGA07_output.xlsx
BSB273BC05HSOXYLGA08_output.xlsx
BSB273BC05HSOXYLGA09_output.xlsx
BSB273BC05HSOXYLGA10_output.xlsx
BSB273BC05HSOXYLGA11_output.xlsx
BSB273BC05HSOXYLGA12_output.xlsx
BSB273BC05HSOXYLGA13_output.xlsx
BSB273BC05HSOXYLGA14_output.xlsx
BSB273BC06HSOXYLGA01_output.xlsx
BSB273BC06HSOXYLGA02_output.xlsx
BSB273BC06

BSB273CC13HSOXYLGA12_output.xlsx
BSB273CC13HSOXYLGA14_output.xlsx
BSB273DC04HSOXYLGA01_output.xlsx
BSB273DC04HSOXYLGA02_output.xlsx
BSB273DC04HSOXYLGA03_output.xlsx
BSB273DC04HSOXYLGA04_output.xlsx
BSB273DC04HSOXYLGA05_output.xlsx
BSB273DC04HSOXYLGA06_output.xlsx
BSB273DC04HSOXYLGA07_output.xlsx
BSB273DC04HSOXYLGA08_output.xlsx
BSB273DC04HSOXYLGA09_output.xlsx
BSB273DC04HSOXYLGA10_output.xlsx
BSB273DC04HSOXYLGA11_output.xlsx
BSB273DC04HSOXYLGA12_output.xlsx
BSB273DC04HSOXYLGA13_output.xlsx
BSB273DC04HSOXYLGA14_output.xlsx
BSB273DC05HSOXYLGA01_output.xlsx
BSB273DC05HSOXYLGA02_output.xlsx
BSB273DC05HSOXYLGA03_output.xlsx
BSB273DC05HSOXYLGA04_output.xlsx
BSB273DC05HSOXYLGA05_output.xlsx
BSB273DC05HSOXYLGA06_output.xlsx
BSB273DC05HSOXYLGA07_output.xlsx
BSB273DC05HSOXYLGA08_output.xlsx
BSB273DC05HSOXYLGA09_output.xlsx
BSB273DC05HSOXYLGA10_output.xlsx
BSB273DC05HSOXYLGA11_output.xlsx
BSB273DC05HSOXYLGA12_output.xlsx
BSB273DC05HSOXYLGA13_output.xlsx
BSB273DC05HSOXYLGA14_output.xlsx
BSB273DC06

BSB273EC13HSOXYLGA10_output.xlsx
BSB273EC13HSOXYLGA11_output.xlsx
BSB273EC13HSOXYLGA12_output.xlsx
BSB273EC13HSOXYLGA13_output.xlsx
BSB273EC13HSOXYLGA14_output.xlsx
MED1110C01HSOXYLGA01_output.xlsx
MED1110C01HSOXYLGA02_output.xlsx
MED1110C01HSOXYLGA03_output.xlsx
MED1110C01HSOXYLGA04_output.xlsx
MED1110C01HSOXYLGA05_output.xlsx
MED1110C01HSOXYLGA07_output.xlsx
MED1110C01HSOXYLGA08_output.xlsx
MED1110C01HSOXYLGA09_output.xlsx
MED1110C01HSOXYLGA10_output.xlsx
MED1110C01HSOXYLGA11_output.xlsx
MED1110C01HSOXYLGA12_output.xlsx
MED1110C01HSOXYLGA13_output.xlsx
MED1110C01HSOXYLGA14_output.xlsx
MED1110C03HSOXYLGA01_output.xlsx
MED1110C03HSOXYLGA02_output.xlsx
MED1110C03HSOXYLGA03_output.xlsx
MED1110C03HSOXYLGA04_output.xlsx
MED1110C03HSOXYLGA05_output.xlsx
MED1110C03HSOXYLGA06_output.xlsx
MED1110C03HSOXYLGA07_output.xlsx
MED1110C03HSOXYLGA08_output.xlsx
MED1110C03HSOXYLGA09_output.xlsx
MED1110C03HSOXYLGA10_output.xlsx
MED1110C03HSOXYLGA11_output.xlsx
MED1110C03HSOXYLGA12_output.xlsx
MED1110C03

MTF134EC16HSOXYLGA11_output.xlsx
MTF134EC16HSOXYLGA12_output.xlsx
MTF134EC16HSOXYLGA13_output.xlsx
MTF134EC16HSOXYLGA14_output.xlsx


In [200]:
df_rawcalcs = pd.concat(calcs, ignore_index=True)
df_rawcalcs.head()

,Subject,meanInterval,stdInterval,modeInterval,meanNumRewards,numBurst,maxBurst,pctRewards,file_id
0,F401,652.64,972.14,23.0,2.18,17,3,73.0,LGA01
1,F402,1011.34,1103.48,29.0,2.33,6,3,33.0,LGA01
2,F403,1945.30,1333.18,2488.0,2.00,4,2,38.0,LGA01
3,F404,988.86,1626.02,6055.0,2.00,8,2,44.0,LGA01
4,F405,587.65,892.74,21.0,3.38,13,6,61.0,LGA01


In [369]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [209]:
df_out = df_rawcalcs[df_rawcalcs['Subject']!='0']
df_out.to_csv('calculations_LGA01-14_uncompressed.csv')

In [373]:
df_out.head()

,Subject,meanInterval,stdInterval,modeInterval,meanNumRewards,numBurst,maxBurst,pctRewards,file_id
0,F401,652.64,972.14,23.0,2.18,17,3,73.0,LGA01
1,F402,1011.34,1103.48,29.0,2.33,6,3,33.0,LGA01
2,F403,1945.30,1333.18,2488.0,2.00,4,2,38.0,LGA01
3,F404,988.86,1626.02,6055.0,2.00,8,2,44.0,LGA01
4,F405,587.65,892.74,21.0,3.38,13,6,61.0,LGA01


In [376]:
temp = df_rawcalcs[df_rawcalcs.Subject == 'F1001']
temp = temp.reset_index(drop=True)
temp.sort_values(['Subject','file_id'],inplace=True)
temp

,Subject,meanInterval,stdInterval,modeInterval,meanNumRewards,numBurst,maxBurst,pctRewards,file_id
0,F1001,NaN,NaN,NaN,0.0,0,0,0.0,LGA01
1,F1001,NaN,NaN,NaN,0.0,0,0,NaN,LGA02
2,F1001,NaN,NaN,NaN,0.0,0,0,NaN,LGA03
3,F1001,NaN,NaN,NaN,0.0,0,0,0.0,LGA04
4,F1001,NaN,NaN,NaN,0.0,0,0,NaN,LGA05
5,F1001,NaN,NaN,NaN,0.0,0,0,0.0,LGA06
6,F1001,NaN,NaN,NaN,0.0,0,0,NaN,LGA07
7,F1001,NaN,NaN,NaN,0.0,0,0,NaN,LGA08
8,F1001,NaN,NaN,NaN,0.0,0,0,NaN,LGA09
9,F1001,NaN,NaN,NaN,0.0,0,0,0.0,LGA10


In [377]:
if len(temp) > 14:
    dup_idx = temp.duplicated(subset=['file_id']).to_numpy()
    dup_files = temp.file_id[dup_idx].values
    for i in dup_files:
        dup_data = temp[temp['file_id'] == i]
        nan_counts = dup_data.isnull().sum(axis=1)
        rmv_idx = nan_counts.idxmax()
        temp = temp.drop(rmv_idx).reset_index(drop=True)

In [378]:
temp = temp.set_index('file_id')

In [354]:
row1 = temp.iloc[:,1:].unstack().to_frame().sort_index(level=1).T
row1.insert(loc=0, column='Subject', value=temp.Subject.unique())
row1

,Subject,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval
file_id,,LGA01,LGA01,LGA01,LGA01,LGA01,LGA01,LGA01,LGA02,LGA02,LGA02,LGA02,LGA02,LGA02,LGA02,LGA03,LGA03,LGA03,LGA03,LGA03,LGA03,LGA03,LGA04,LGA04,LGA04,LGA04,LGA04,LGA04,LGA04,LGA05,LGA05,LGA05,LGA05,LGA05,LGA05,LGA05,LGA07,LGA07,LGA07,LGA07,LGA07,LGA07,LGA07,LGA08,LGA08,LGA08,LGA08,LGA08,LGA08,LGA08,LGA09,LGA09,LGA09,LGA09,LGA09,LGA09,LGA09,LGA10,LGA10,LGA10,LGA10,LGA10,LGA10,LGA10,LGA11,LGA11,LGA11,LGA11,LGA11,LGA11,LGA11,LGA12,LGA12,LGA12,LGA12,LGA12,LGA12,LGA12,LGA13,LGA13,LGA13,LGA13,LGA13,LGA13,LGA13,LGA14,LGA14,LGA14,LGA14,LGA14,LGA14,LGA14
0,F402,3.0,1011.34,2.33,29.0,6.0,33.0,1103.48,2.0,3409.0,2.0,4056.0,2.0,31.0,2525.62,3.0,969.55,2.33,26.0,9.0,47.0,1041.71,0.0,3897.86,0.0,5636.0,0.0,0.0,4042.38,2.0,3497.73,2.0,3572.0,4.0,67.0,3138.74,2.0,5537.5,2.0,12347.0,1.0,40.0,4398.35,3.0,879.98,2.12,332.0,8.0,41.0,1341.98,4.0,2780.87,3.5,25.0,2.0,44.0,2794.2,3.0,885.48,2.14,24.0,7.0,50.0,2181.2,4.0,448.39,2.17,27.0,18.0,65.0,581.95,2.0,2447.0,2.0,48.0,3.0,43.0,2739.81,4.0,438.33,2.47,22.0,17.0,46.0,651.86,6.0,501.59,2.88,27.0,24.0,79.0,1130.69


In [347]:
row2 = temp.iloc[:,1:].unstack().to_frame().sort_index(level=1).T
row2.insert(loc=0, column='Subject', value=temp.Subject.unique())
row2

,Subject,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval
file_id,,LGA01,LGA01,LGA01,LGA01,LGA01,LGA01,LGA01,LGA02,LGA02,LGA02,LGA02,LGA02,LGA02,LGA02,LGA03,LGA03,LGA03,LGA03,LGA03,LGA03,LGA03,LGA04,LGA04,LGA04,LGA04,LGA04,LGA04,LGA04,LGA05,LGA05,LGA05,LGA05,LGA05,LGA05,LGA05,LGA06,LGA06,LGA06,LGA06,LGA06,LGA06,LGA06,LGA07,LGA07,LGA07,LGA07,LGA07,LGA07,LGA07,LGA08,LGA08,LGA08,LGA08,LGA08,LGA08,LGA08,LGA09,LGA09,LGA09,LGA09,LGA09,LGA09,LGA09,LGA10,LGA10,LGA10,LGA10,LGA10,LGA10,LGA10,LGA11,LGA11,LGA11,LGA11,LGA11,LGA11,LGA11,LGA12,LGA12,LGA12,LGA12,LGA12,LGA12,LGA12,LGA13,LGA13,LGA13,LGA13,LGA13,LGA13,LGA13,LGA14,LGA14,LGA14,LGA14,LGA14,LGA14,LGA14
0,F502,2.0,9813.67,2.0,25523.0,1.0,50.0,11215.87,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,14830.5,0.0,8051.0,0.0,0.0,6779.5,0.0,NaN,0.0,NaN,0.0,NaN,NaN,2.0,1391.71,2.0,2790.0,5.0,45.0,2326.05,3.0,978.86,2.22,52.0,9.0,44.0,1049.32,3.0,707.22,2.38,23.0,8.0,46.0,984.37,3.0,2018.0,2.2,21.0,5.0,50.0,2459.37,2.0,3168.77,2.0,1008.0,1.0,14.0,3841.74,3.0,905.16,2.43,21.0,7.0,44.0,1105.63,2.0,1081.36,2.0,845.0,2.0,11.0,1169.25,3.0,610.3,2.3,23.0,10.0,37.0,496.76,3.0,598.42,2.12,27.0,17.0,53.0,774.31,3.0,1240.76,2.25,48.0,8.0,53.0,2799.79


In [379]:
row3 = temp.iloc[:,1:].unstack().to_frame().sort_index(level=1).T
row3.insert(loc=0, column='Subject', value=temp.Subject.unique())
row3

,Subject,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval
file_id,,LGA01,LGA01,LGA01,LGA01,LGA01,LGA01,LGA01,LGA02,LGA02,LGA02,LGA02,LGA02,LGA02,LGA02,LGA03,LGA03,LGA03,LGA03,LGA03,LGA03,LGA03,LGA04,LGA04,LGA04,LGA04,LGA04,LGA04,LGA04,LGA05,LGA05,LGA05,LGA05,LGA05,LGA05,LGA05,LGA06,LGA06,LGA06,LGA06,LGA06,LGA06,LGA06,LGA07,LGA07,LGA07,LGA07,LGA07,LGA07,LGA07,LGA08,LGA08,LGA08,LGA08,LGA08,LGA08,LGA08,LGA09,LGA09,LGA09,LGA09,LGA09,LGA09,LGA09,LGA10,LGA10,LGA10,LGA10,LGA10,LGA10,LGA10,LGA11,LGA11,LGA11,LGA11,LGA11,LGA11,LGA11,LGA12,LGA12,LGA12,LGA12,LGA12,LGA12,LGA12,LGA13,LGA13,LGA13,LGA13,LGA13,LGA13,LGA13,LGA14,LGA14,LGA14,LGA14,LGA14,LGA14,LGA14
0,F1001,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,28538.0,0.0,28538.0,0.0,0.0,0.0,3.0,3948.0,3.0,36.0,1.0,60.0,6393.96


In [380]:
pd.concat([row1,row2,row3],ignore_index=True).rename(index={'file_id': 'index'})

0       Subject maxBurst                                                  \
file_id            LGA01 LGA02 LGA03 LGA04 LGA05 LGA06 LGA07 LGA08 LGA09   
0          F402      3.0   2.0   3.0   0.0   2.0   NaN   2.0   3.0   4.0   
1          F502      2.0   0.0   0.0   0.0   2.0   3.0   3.0   3.0   2.0   
2         F1001      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

0                                     meanInterval                             \
file_id LGA10 LGA11 LGA12 LGA13 LGA14        LGA01   LGA02     LGA03    LGA04   
0         3.0   4.0   2.0   4.0   6.0      1011.34  3409.0    969.55  3897.86   
1         3.0   2.0   3.0   3.0   3.0      9813.67     NaN  14830.50      NaN   
2         0.0   0.0   0.0   0.0   3.0          NaN     NaN       NaN      NaN   

0                                                                             \
file_id    LGA05   LGA06    LGA07    LGA08    LGA09   LGA10    LGA11   LGA12   
0        3497.73     NaN  5537.50   879.98  2780.87  885.48   448.39  2447.0   
1        1391.71  978.86   707.22  2018.00  3168.77  905.16  1081.36   610.3   
2            NaN     NaN      NaN      NaN      NaN     NaN      NaN     NaN   

0                          meanNumRewards                                      \
file_id     LGA13    LGA14          LGA01 LGA02 LGA03 LGA04 LGA05 LGA06 LGA07   
0          438.33   501.59           2.33   2.0  2.33   0.0   2.0   NaN  2.00   
1          598.42  1240.76           2.00   0.0  0.00   0.0   2.0  2.22  2.38   
2        28538.00  3948.00           0.00   0.0  0.00   0.0   0.0  0.00  0.00   

0                                                 modeInterval          \
file_id LGA08 LGA09 LGA10 LGA11 LGA12 LGA13 LGA14        LGA01   LGA02   
0        2.12   3.5  2.14  2.17   2.0  2.47  2.88         29.0  4056.0   
1        2.20   2.0  2.43  2.00   2.3  2.12  2.25      25523.0     NaN   
2        0.00   0.0  0.00  0.00   0.0  0.00  3.00          NaN     NaN   

0                                                                           \
file_id   LGA03   LGA04   LGA05 LGA06    LGA07  LGA08   LGA09 LGA10  LGA11   
0          26.0  5636.0  3572.0   NaN  12347.0  332.0    25.0  24.0   27.0   
1        8051.0     NaN  2790.0  52.0     23.0   21.0  1008.0  21.0  845.0   
2           NaN     NaN     NaN   NaN      NaN    NaN     NaN   NaN    NaN   

0                            numBurst                                      \
file_id LGA12    LGA13 LGA14    LGA01 LGA02 LGA03 LGA04 LGA05 LGA06 LGA07   
0        48.0     22.0  27.0      6.0   2.0   9.0   0.0   4.0   NaN   1.0   
1        23.0     27.0  48.0      1.0   0.0   0.0   0.0   5.0   9.0   8.0   
2         NaN  28538.0  36.0      0.0   0.0   0.0   0.0   0.0   0.0   0.0   

0                                                 pctRewards              \
file_id LGA08 LGA09 LGA10 LGA11 LGA12 LGA13 LGA14      LGA01 LGA02 LGA03   
0         8.0   2.0   7.0  18.0   3.0  17.0  24.0       33.0  31.0  47.0   
1         5.0   1.0   7.0   2.0  10.0  17.0   8.0       50.0   NaN   0.0   
2         0.0   0.0   0.0   0.0   0.0   0.0   1.0        0.0   NaN   NaN   

0                                                                          \
file_id LGA04 LGA05 LGA06 LGA07 LGA08 LGA09 LGA10 LGA11 LGA12 LGA13 LGA14   
0         0.0  67.0   NaN  40.0  41.0  44.0  50.0  65.0  43.0  46.0  79.0   
1         NaN  45.0  44.0  46.0  50.0  14.0  44.0  11.0  37.0  53.0  53.0   
2         0.0   NaN   0.0   NaN   NaN   NaN   0.0   NaN   NaN   0.0  60.0   

0       stdInterval                                                        \
file_id       LGA01    LGA02    LGA03    LGA04    LGA05    LGA06    LGA07   
0           1103.48  2525.62  1041.71  4042.38  3138.74      NaN  4398.35   
1          11215.87      NaN  6779.50      NaN  2326.05  1049.32   984.37   
2               NaN      NaN      NaN      NaN      NaN      NaN      NaN   

0                                                                      
file_id    LGA08    LGA09    LGA1

In [381]:
rows = []
subs = [i for i in df_rawcalcs.Subject.unique() if i != '0' and i != 'C07HSOXYLGA09']
for s in subs:
    temp = df_rawcalcs[df_rawcalcs.Subject == s]
    temp = temp.reset_index(drop=True)
    temp.sort_values(['Subject','file_id'],inplace=True)
    
    if len(temp) > 14:
        dup_idx = temp.duplicated(subset=['file_id']).to_numpy()
        dup_files = temp.file_id[dup_idx].values
        for i in dup_files:
            dup_data = temp[temp['file_id'] == i]
            nan_counts = dup_data.isnull().sum(axis=1)
            rmv_idx = nan_counts.idxmax()
            temp = temp.drop(rmv_idx).reset_index(drop=True)
            
    temp = temp.set_index('file_id')
    temp_row = temp.iloc[:,1:].unstack().to_frame().sort_index(level=1).T
    temp_row.insert(loc=0, column='Subject', value=temp.Subject.unique())
    temp_row = temp_row.reset_index(drop=True)
    rows.append(temp_row)

In [388]:
len(rows)

891

In [386]:
type(rows[0])

pandas.core.frame.DataFrame

In [429]:
pd.concat(rows[:31],ignore_index=True)

0       Subject maxBurst                                                  \
file_id            LGA01 LGA02 LGA03 LGA04 LGA05 LGA06 LGA07 LGA08 LGA09   
0          F401      3.0   2.0   3.0   3.0   2.0   3.0   2.0   4.0   5.0   
1          F402      3.0   2.0   3.0   0.0   2.0   NaN   2.0   3.0   4.0   
2          F403      2.0   2.0   3.0   2.0   2.0   4.0   2.0   2.0   2.0   
3          F404      2.0   3.0   3.0   4.0   3.0   NaN   4.0   4.0   4.0   
4          F405      6.0   0.0   2.0   5.0   3.0   NaN   4.0   3.0   4.0   
..          ...      ...   ...   ...   ...   ...   ...   ...   ...   ...   
26         F511      0.0   0.0   2.0   0.0   4.0   5.0   2.0   2.0   5.0   
27         F512      3.0   4.0   4.0   5.0   5.0   5.0   5.0   5.0   6.0   
28         F513      4.0   3.0   2.0   3.0   2.0   4.0   3.0   3.0   3.0   
29         F514      0.0   0.0   0.0   0.0   0.0   0.0   2.0   2.0   0.0   
30         F516      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

0                                     meanInterval                      \
file_id LGA10 LGA11 LGA12 LGA13 LGA14        LGA01     LGA02     LGA03   
0         3.0   4.0   4.0   5.0   3.0       652.64    618.26    505.85   
1         3.0   4.0   2.0   4.0   6.0      1011.34   3409.00    969.55   
2         2.0   2.0   3.0   3.0   3.0      1945.30   1483.00    686.46   
3         5.0   5.0   4.0   5.0   4.0       988.86    624.13    834.39   
4         4.0   3.0   4.0   4.0   3.0       587.65  10884.00   7181.20   
..        ...   ...   ...   ...   ...          ...       ...       ...   
26        6.0   6.0   3.0   6.0   6.0      8702.50   2825.08   2548.56   
27        5.0   6.0   3.0   2.0   6.0       827.67    241.10    506.85   
28        2.0   2.0   4.0   4.0   3.0       443.00   2405.12  10322.25   
29        0.0   0.0   0.0   0.0   0.0     12512.00  20378.50       NaN   
30        0.0   2.0   3.0   3.0   2.0          NaN       NaN       NaN   

0                                                                      \
file_id    LGA04    LGA05   LGA06    LGA07    LGA08     LGA09   LGA10   
0         922.20   602.54  172.05   398.12   431.00    490.03  411.97   
1        3897.86  3497.73     NaN  5537.50   879.98   2780.87  885.48   
2        1792.04  1105.63  236.69  1737.04  1125.13    744.50  668.69   
3         509.23  1099.32     NaN   599.62   402.63    296.28  252.77   
4         506.65  3649.75     NaN   463.26   497.42   1513.62  285.57   
..           ...      ...     ...      ...      ...       ...     ...   
26       2254.13   775.08  777.56  1742.95  1649.20    855.34  734.02   
27        264.31   342.88  289.31   644.28   288.01    370.84  198.76   
28       1024.08   799.92  518.94   881.82  1059.89    870.38  612.19   
29       9181.00  9112.75  138.00  3072.79  7539.67       NaN     NaN   
30           NaN      NaN     NaN      NaN      NaN  23686.00     NaN   

0                                           meanNumRewards                    \
file_id    LGA11    LGA12    LGA13    LGA14          LGA01 LGA02 LGA03 LGA04   
0         393.97   322.64   323.17   359.99           2.18  2.00  2.19  2.15   
1         448.39  2447.00   438.33   501.59           2.33  2.00  2.33  0.00   
2         449.37   479.85   490.76   517.37           2.00  2.00  2.10  2.00   
3         327.90   327.15   327.00   470.84           2.00  2.08  2.10  2.56   
4        2023.90   428.62   372.03   385.48           3.38  0.00  2.00  3.15   
..           ...      ...      ...      ...            ...   ...   ...   ...   
26        484.91  1277.75   228.78   299.89           0.00  0.00  2.00  0.00   
27        215.83  1127.86  1533.00   206.09           2.33  2.30  2.62  2.68   
28        604.38   356.23   893.34   412.70           2.18  2.20  2.00  2.18   
29       5410.00      NaN  4363.83  9185.75           0.00  0.00  0.00  0.00   
30        587.95  1144.70  1080.63   911.64           0.00  0.00  0.00  0.00   

0                                                        

In [410]:
rows[30]

,Subject,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval
file_id,,LGA01,LGA01,LGA01,LGA01,LGA01,LGA01,LGA01,LGA02,LGA02,LGA02,LGA02,LGA02,LGA02,LGA02,LGA03,LGA03,LGA03,LGA03,LGA03,LGA03,LGA03,LGA04,LGA04,LGA04,LGA04,LGA04,LGA04,LGA04,LGA05,LGA05,LGA05,LGA05,LGA05,LGA05,LGA05,LGA06,LGA06,LGA06,LGA06,LGA06,LGA06,LGA06,LGA07,LGA07,LGA07,LGA07,LGA07,LGA07,LGA07,LGA08,LGA08,LGA08,LGA08,LGA08,LGA08,LGA08,LGA09,LGA09,LGA09,LGA09,LGA09,LGA09,LGA09,LGA10,LGA10,LGA10,LGA10,LGA10,LGA10,LGA10,LGA11,LGA11,LGA11,LGA11,LGA11,LGA11,LGA11,LGA12,LGA12,LGA12,LGA12,LGA12,LGA12,LGA12,LGA13,LGA13,LGA13,LGA13,LGA13,LGA13,LGA13,LGA14,LGA14,LGA14,LGA14,LGA14,LGA14,LGA14
0,F516,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,23686.0,0.0,23686.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,2.0,587.95,2.0,208.0,4.0,35.0,517.35,3.0,1144.7,2.25,85.0,4.0,39.0,1269.67,3.0,1080.63,2.33,21.0,3.0,35.0,1159.92,2.0,911.64,2.0,1094.0,3.0,12.0,484.61


In [415]:
rows[30]

,Subject,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval
file_id,,LGA01,LGA01,LGA01,LGA01,LGA01,LGA01,LGA01,LGA02,LGA02,LGA02,LGA02,LGA02,LGA02,LGA02,LGA03,LGA03,LGA03,LGA03,LGA03,LGA03,LGA03,LGA04,LGA04,LGA04,LGA04,LGA04,LGA04,LGA04,LGA05,LGA05,LGA05,LGA05,LGA05,LGA05,LGA05,LGA06,LGA06,LGA06,LGA06,LGA06,LGA06,LGA06,LGA07,LGA07,LGA07,LGA07,LGA07,LGA07,LGA07,LGA08,LGA08,LGA08,LGA08,LGA08,LGA08,LGA08,LGA09,LGA09,LGA09,LGA09,LGA09,LGA09,LGA09,LGA10,LGA10,LGA10,LGA10,LGA10,LGA10,LGA10,LGA11,LGA11,LGA11,LGA11,LGA11,LGA11,LGA11,LGA12,LGA12,LGA12,LGA12,LGA12,LGA12,LGA12,LGA13,LGA13,LGA13,LGA13,LGA13,LGA13,LGA13,LGA14,LGA14,LGA14,LGA14,LGA14,LGA14,LGA14
0,F516,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,23686.0,0.0,23686.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,2.0,587.95,2.0,208.0,4.0,35.0,517.35,3.0,1144.7,2.25,85.0,4.0,39.0,1269.67,3.0,1080.63,2.33,21.0,3.0,35.0,1159.92,2.0,911.64,2.0,1094.0,3.0,12.0,484.61


In [416]:
rows[31]

0       Subject maxBurst meanInterval meanNumRewards modeInterval numBurst  \
file_id            LGA01        LGA01          LGA01        LGA01    LGA01   
0          F535      4.0       1709.7            2.5         22.0      6.0   

0       pctRewards stdInterval maxBurst meanInterval meanNumRewards  \
file_id      LGA01       LGA01    LGA02        LGA02          LGA02   
0             62.0     4436.33      0.0          NaN            0.0   

0       modeInterval numBurst pctRewards stdInterval maxBurst meanInterval  \
file_id        LGA02    LGA02      LGA02       LGA02    LGA03        LGA03   
0                NaN      0.0        0.0         NaN      0.0      26004.0   

0       meanNumRewards modeInterval numBurst pctRewards stdInterval maxBurst  \
file_id          LGA03        LGA03    LGA03      LGA03       LGA03    LGA04   
0                  0.0      26004.0      0.0        0.0         0.0      2.0   

0             meanInterval       meanNumRewards       modeInterval        \
file_id LGA04        LGA04 LGA04          LGA04 LGA04        LGA04 LGA04   
0         0.0      1686.88   NaN            2.0   0.0       1148.0   NaN   

0       numBurst       pctRewards       stdInterval       maxBurst  \
file_id    LGA04 LGA04      LGA04 LGA04       LGA04 LGA04    LGA05   
0            5.0   0.0       38.0   NaN     1226.06   NaN      2.0   

0       meanInterval meanNumRewards modeInterval numBurst pctRewards  \
file_id        LGA05          LGA05        LGA05    LGA05      LGA05   
0             748.63            2.0        434.0      1.0        4.0   

0       stdInterval maxBurst meanInterval meanNumRewards modeInterval  \
file_id       LGA05    LGA06        LGA06          LGA06        LGA06   
0            355.13      3.0      1373.23           2.17         35.0   

0       numBurst pctRewards stdInterval maxBurst meanInterval meanNumRewards  \
file_id    LGA06      LGA06       LGA06    LGA07        LGA07          LGA07   
0            6.0       41.0     1986.41      3.0       957.51           2.25   

0       modeInterval numBurst pctRewards stdInterval maxBurst meanInterval  \
file_id        LGA07    LGA07      LGA07       LGA07    LGA08        LGA08   
0               27.0      8.0       39.0      907.33      3.0      1003.51   

0       meanNumRewards modeInterval numBurst pctRewards stdInterval maxBurst  \
file_id          LGA08        LGA08    LGA08      LGA08       LGA08    LGA09   
0                  2.2        149.0     10.0       55.0      1119.3      2.0   

0       meanInterval meanNumRewards modeInterval numBurst pctRewards  \
file_id        LGA09          LGA09        LGA09    LGA09      LGA09   
0            1198.69            2.0         21.0      5.0       33.0   

0       stdInterval maxBurst meanInterval meanNumRewards modeInterval  \
file_id       LGA09    LGA10        LGA10          LGA10        LGA10   
0            1422.2      2.0       603.24            2.0        641.0   

0       numBurst pctRewards stdInterval maxBurst meanInterval meanNumRewards  \
file_id    LGA10      LGA10       LGA10    LGA11        LGA11          LGA11   
0            4.0       11.0      258.21      4.0       500.96           2.44   

0       modeInterval numBurst pctRewards stdInterval maxBurst meanInterval  \
file_id        LGA11    LGA11      LGA11       LGA11    LGA12        LGA12   
0               32.0     18.0       51.0      519.17      3.0       386.65   

0       meanNumRewards modeInterval numBurst pctRewards stdInterval maxBurst  \
file_id          LGA12        LGA12    LGA12      LGA12       LGA12    LGA13   
0                 2.13        401.0     31.0       59.0      315.47      4.0   

0       meanInterval meanNumRewards modeInterval numBurst pctRewards  \
file_id        LGA13          LGA13        LGA13    LGA13      LGA13   
0             620.26           2.24         25.0     17.0       58.0   

0       stdInterval  
file_id       LGA13  
0            1379.5

In [417]:
rows[32]

,Subject,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval,maxBurst,meanInterval,meanNumRewards,modeInterval,numBurst,pctRewards,stdInterval
file_id,,LGA01,LGA01,LGA01,LGA01,LGA01,LGA01,LGA01,LGA02,LGA02,LGA02,LGA02,LGA02,LGA02,LGA02,LGA03,LGA03,LGA03,LGA03,LGA03,LGA03,LGA03,LGA04,LGA04,LGA04,LGA04,LGA04,LGA04,LGA04,LGA05,LGA05,LGA05,LGA05,LGA05,LGA05,LGA05,LGA06,LGA06,LGA06,LGA06,LGA06,LGA06,LGA06,LGA07,LGA07,LGA07,LGA07,LGA07,LGA07,LGA07,LGA08,LGA08,LGA08,LGA08,LGA08,LGA08,LGA08,LGA09,LGA09,LGA09,LGA09,LGA09,LGA09,LGA09,LGA10,LGA10,LGA10,LGA10,LGA10,LGA10,LGA10,LGA11,LGA11,LGA11,LGA11,LGA11,LGA11,LGA11,LGA12,LGA12,LGA12,LGA12,LGA12,LGA12,LGA12,LGA13,LGA13,LGA13,LGA13,LGA13,LGA13,LGA13,LGA14,LGA14,LGA14,LGA14,LGA14,LGA14,LGA14
0,F538,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,4064.0,0.0,4064.0,0.0,0.0,0.0,3.0,1474.09,2.14,21414.0,7.0,65.0,4404.15,0.0,NaN,0.0,NaN,0.0,0.0,NaN,4.0,1078.46,2.43,21.0,14.0,85.0,3371.22,2.0,788.62,2.0,27.0,11.0,51.0,633.03,5.0,976.24,3.0,27.0,8.0,69.0,1652.08,3.0,1673.38,2.09,31.0,11.0,92.0,2327.05,4.0,571.64,2.27,26.0,15.0,47.0,476.87,4.0,901.98,2.38,50.0,8.0,41.0,1155.22,3.0,523.71,2.07,32.0,14.0,45.0,560.97,3.0,1702.16,2.11,26.0,9.0,73.0,4599.47,4.0,650.42,2.27,26.0,15.0,52.0,644.64


In [478]:
rows[164]

0       Subject maxBurst meanInterval meanNumRewards modeInterval numBurst  \
file_id            LGA01        LGA01          LGA01        LGA01    LGA01   
0          F518      5.0      1229.83            2.5         21.0      8.0   

0       pctRewards stdInterval maxBurst meanInterval meanNumRewards  \
file_id      LGA01       LGA01    LGA02        LGA02          LGA02   
0             80.0     4345.93      3.0      3433.73           2.33   

0       modeInterval numBurst pctRewards stdInterval maxBurst meanInterval  \
file_id        LGA02    LGA02      LGA02       LGA02    LGA03        LGA03   
0               22.0      3.0       58.0     8395.18      2.0       1900.0   

0       meanNumRewards modeInterval numBurst pctRewards stdInterval maxBurst  \
file_id          LGA03        LGA03    LGA03      LGA03       LGA03    LGA04   
0                  2.0         25.0      2.0       19.0     2879.32      2.0   

0       meanInterval meanNumRewards modeInterval numBurst pctRewards  \
file_id        LGA04          LGA04        LGA04    LGA04      LGA04   
0            2681.88            2.0       1920.0      2.0       24.0   

0       stdInterval maxBurst meanInterval meanNumRewards modeInterval  \
file_id       LGA04    LGA05        LGA05          LGA05        LGA05   
0           3691.13      3.0      1054.22           2.12         84.0   

0       numBurst pctRewards stdInterval maxBurst meanInterval meanNumRewards  \
file_id    LGA05      LGA05       LGA05    LGA06        LGA06          LGA06   
0            8.0       41.0     1517.82      2.0      1550.46            2.0   

0       modeInterval numBurst pctRewards stdInterval maxBurst meanInterval  \
file_id        LGA06    LGA06      LGA06       LGA06    LGA07        LGA07   
0              171.0      3.0       24.0     2063.02      2.0      1212.74   

0       meanNumRewards modeInterval numBurst pctRewards stdInterval maxBurst  \
file_id          LGA07        LGA07    LGA07      LGA07       LGA07    LGA08   
0                  2.0        552.0      3.0       17.0      881.79      2.0   

0       meanInterval meanNumRewards modeInterval numBurst pctRewards  \
file_id        LGA08          LGA08        LGA08    LGA08      LGA08   
0             774.76            2.0        601.0      4.0       21.0   

0       stdInterval maxBurst meanInterval meanNumRewards modeInterval  \
file_id       LGA08    LGA09        LGA09          LGA09        LGA09   
0            559.87      2.0      1049.73            2.0         29.0   

0       numBurst pctRewards stdInterval maxBurst meanInterval meanNumRewards  \
file_id    LGA09      LGA09       LGA09    LGA10        LGA10          LGA10   
0            5.0       32.0      761.87      2.0      1171.25            2.0   

0       modeInterval numBurst pctRewards stdInterval maxBurst        \
file_id        LGA10    LGA10      LGA10       LGA10    LGA11 LGA11   
0              731.0      3.0       16.0     1920.61      2.0   2.0   

0       meanInterval         meanNumRewards       modeInterval         \
file_id        LGA11   LGA11          LGA11 LGA11        LGA11  LGA11   
0            1066.05  654.58            2.0   2.0         27.0  897.0   

0       numBurst       pctRewards       stdInterval         maxBurst  \
file_id    LGA11 LGA11      LGA11 LGA11       LGA11   LGA11    LGA13   
0            4.0   4.0       20.0  12.0      540.63  302.17      3.0   

0       meanInterval meanNumRewards modeInterval numBurst pctRewards  \
file_id        LGA13          LGA13        LGA13    LGA13      LGA13   
0             588.19            2.2        789.0      5.0       16.0   

0       stdInterval maxBurst meanInterval meanNumRewards modeInterval  \
file_id       LGA13    LGA14        LGA14          LGA14        LGA14   
0            334.04      4.0       643.76           2.29         69.0   

0       numBurst pctRewards stdInterval  
file_id    LGA14      LGA14       LGA14  
0            7.0       24.0      775.06

In [509]:
pd.concat(rows[200:210],ignore_index=True)

0       Subject maxBurst                                                  \
file_id            LGA02 LGA03 LGA04 LGA05 LGA06 LGA07 LGA08 LGA09 LGA10   
0          M620      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   3.0   
1          M621      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0   
2          M622      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   0.0   
3          M623      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   3.0   
4          M624      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   6.0   
5          F717      2.0   3.0   3.0   2.0   2.0   4.0   3.0   4.0   2.0   
6          F718      4.0   4.0   3.0   3.0   3.0   3.0   3.0   3.0   4.0   
7          F719      2.0   0.0   0.0   0.0   0.0   2.0   0.0   0.0   NaN   
8          F720      3.0   2.0   2.0   2.0   2.0   2.0   2.0   3.0   2.0   
9          F722      5.0   2.0   0.0   0.0   NaN   NaN   NaN   NaN   NaN   

0                                     meanInterval                             \
file_id LGA11 LGA12 LGA13 LGA14 LGA15        LGA02    LGA03    LGA04    LGA05   
0         NaN   NaN   NaN   NaN   NaN          NaN      NaN      NaN      NaN   
1         NaN   NaN   NaN   NaN   NaN          NaN      NaN      NaN      NaN   
2         NaN   NaN   NaN   NaN   NaN          NaN      NaN      NaN      NaN   
3         NaN   NaN   NaN   NaN   NaN          NaN      NaN      NaN      NaN   
4         NaN   NaN   NaN   NaN   NaN          NaN      NaN      NaN      NaN   
5         4.0   4.0   4.0   NaN   2.0       677.60  1015.92   800.48  2087.24   
6         4.0   2.0   2.0   3.0   2.0      1372.65   542.60  1278.33  1413.34   
7         NaN   NaN   NaN   NaN   NaN     10536.00      NaN  3408.00  7204.50   
8         2.0   2.0   2.0   3.0   2.0      1168.77  2877.38   939.38   989.13   
9         NaN   NaN   NaN   NaN   NaN      1018.80  5634.00      NaN      NaN   

0                                                                       \
file_id    LGA06    LGA07    LGA08    LGA09    LGA10    LGA11    LGA12   
0            NaN      NaN      NaN      NaN  2015.00      NaN      NaN   
1            NaN      NaN      NaN      NaN   419.10      NaN      NaN   
2            NaN      NaN      NaN      NaN   149.00      NaN      NaN   
3            NaN      NaN      NaN      NaN   538.83      NaN      NaN   
4            NaN      NaN      NaN      NaN   284.87      NaN      NaN   
5        1564.00  1106.39  1060.54   781.70   789.28   776.33   556.01   
6        1346.87  2012.89  2307.69  1410.77   775.87  1341.97  1185.48   
7        3459.00  1897.67      NaN      NaN      NaN      NaN      NaN   
8        1219.50   849.96   530.23   517.97   551.49   496.06   513.22   
9            NaN      NaN      NaN      NaN      NaN      NaN      NaN   

0                                  meanNumRewards                          \
file_id    LGA13    LGA14    LGA15          LGA02 LGA03 LGA04 LGA05 LGA06   
0            NaN      NaN      NaN            NaN   NaN   NaN   NaN   NaN   
1            NaN      NaN      NaN            NaN   NaN   NaN   NaN   NaN   
2            NaN      NaN      NaN            NaN   NaN   NaN   NaN   NaN   
3            NaN      NaN      NaN            NaN   NaN   NaN   NaN   NaN   
4            NaN      NaN      NaN            NaN   NaN   NaN   NaN   NaN   
5         464.45      NaN  1040.98           2.00  2.08  2.11  2.00  2.00   
6        2311.47  1077.22  1093.95           2.33  2.50  2.14  2.29  2.22   
7            NaN      NaN      NaN           2.00  0.00  0.00  0.00  0.00   
8         433.40   386.82   501.27           2.33  2.00  2.00  2.00  2.00   
9            NaN      NaN      NaN           3.38  2.00  0.00  0.00   NaN   

0                                                             modeInterval  \
file_id LGA07 LGA08 LGA09 LGA10 LGA11 LGA12 LGA13 LGA14 LGA15        LGA02   
0         NaN   NaN   NaN  3.00   NaN   NaN   NaN   NaN   NaN          NaN   
1         NaN   NaN   NaN  2.00   NaN   NaN   NaN   NaN   NaN     

In [538]:
rows[814]

0       Subject maxBurst meanInterval meanNumRewards modeInterval numBurst  \
file_id            LGA01        LGA01          LGA01        LGA01    LGA01   
0         M1460      2.0      2059.84            2.0       1246.0      3.0   

0       pctRewards stdInterval maxBurst meanInterval meanNumRewards  \
file_id      LGA01       LGA01    LGA02        LGA02          LGA02   
0             30.0     2603.42      2.0      2225.53            2.0   

0       modeInterval numBurst pctRewards stdInterval maxBurst        \
file_id        LGA02    LGA02      LGA02       LGA02    LGA03 LGA03   
0               29.0      1.0       12.0     3303.08      3.0   3.0   

0       meanInterval          meanNumRewards       modeInterval          \
file_id        LGA03    LGA03          LGA03 LGA03        LGA03   LGA03   
0            1132.03  1132.03           2.17  2.17       2146.0  2146.0   

0       numBurst       pctRewards       stdInterval          maxBurst  \
file_id    LGA03 LGA03      LGA03 LGA03       LGA03    LGA03    LGA04   
0            6.0   6.0       33.0  33.0     1034.66  1034.66      2.0   

0       meanInterval meanNumRewards modeInterval numBurst pctRewards  \
file_id        LGA04          LGA04        LGA04    LGA04      LGA04   
0            1375.43            2.0       5786.0      3.0       21.0   

0       stdInterval maxBurst meanInterval meanNumRewards modeInterval  \
file_id       LGA04    LGA05        LGA05          LGA05        LGA05   
0           1535.83      2.0      1100.72            2.0       1214.0   

0       numBurst pctRewards stdInterval maxBurst meanInterval meanNumRewards  \
file_id    LGA05      LGA05       LGA05    LGA06        LGA06          LGA06   
0            2.0       10.0     1349.06      2.0      1185.21            2.0   

0       modeInterval numBurst pctRewards stdInterval maxBurst meanInterval  \
file_id        LGA06    LGA06      LGA06       LGA06    LGA07        LGA07   
0              109.0      3.0       24.0     2051.55      0.0      1379.07   

0       meanNumRewards modeInterval numBurst pctRewards stdInterval maxBurst  \
file_id          LGA07        LGA07    LGA07      LGA07       LGA07    LGA08   
0                  0.0       1180.0      0.0        0.0     1270.63      5.0   

0       meanInterval meanNumRewards modeInterval numBurst pctRewards  \
file_id        LGA08          LGA08        LGA08    LGA08      LGA08   
0             374.41            3.0         21.0     23.0       66.0   

0       stdInterval maxBurst meanInterval meanNumRewards modeInterval  \
file_id       LGA08    LGA09        LGA09          LGA09        LGA09   
0            484.55      5.0       705.95           3.25         20.0   

0       numBurst pctRewards stdInterval maxBurst meanInterval meanNumRewards  \
file_id    LGA09      LGA09       LGA09    LGA10        LGA10          LGA10   
0           12.0       64.0     1501.73      2.0        429.5            2.0   

0       modeInterval numBurst pctRewards stdInterval  
file_id        LGA10    LGA10      LGA10       LGA10  
0              900.0      1.0       40.0      300.79

In [528]:
len(rows)

891

In [539]:
out = pd.concat(rows[:31]+rows[32:131]+rows[132:163] + rows[179:814]+rows[815:]
          ,ignore_index=True)
out

0       Subject maxBurst                                                  \
file_id            LGA01 LGA02 LGA03 LGA04 LGA05 LGA06 LGA07 LGA08 LGA09   
0          F401      3.0   2.0   3.0   3.0   2.0   3.0   2.0   4.0   5.0   
1          F402      3.0   2.0   3.0   0.0   2.0   NaN   2.0   3.0   4.0   
2          F403      2.0   2.0   3.0   2.0   2.0   4.0   2.0   2.0   2.0   
3          F404      2.0   3.0   3.0   4.0   3.0   NaN   4.0   4.0   4.0   
4          F405      6.0   0.0   2.0   5.0   3.0   NaN   4.0   3.0   4.0   
..          ...      ...   ...   ...   ...   ...   ...   ...   ...   ...   
867       M1677      0.0   0.0   0.0   3.0   2.0   3.0   2.0   4.0   2.0   
868       M1678      0.0   0.0   0.0   0.0   0.0   2.0   0.0   0.0   0.0   
869       M1679      2.0   0.0   2.0   0.0   2.0   2.0   3.0   3.0   3.0   
870       M1680      2.0   2.0   2.0   2.0   2.0   2.0   3.0   3.0   3.0   
871       M1685      2.0   2.0   2.0   2.0   3.0   3.0   3.0   2.0   2.0   

0                                           meanInterval                     \
file_id LGA10 LGA11 LGA12 LGA13 LGA14 LGA15        LGA01     LGA02    LGA03   
0         3.0   4.0   4.0   5.0   3.0   NaN       652.64    618.26   505.85   
1         3.0   4.0   2.0   4.0   6.0   NaN      1011.34   3409.00   969.55   
2         2.0   2.0   3.0   3.0   3.0   NaN      1945.30   1483.00   686.46   
3         5.0   5.0   4.0   5.0   4.0   NaN       988.86    624.13   834.39   
4         4.0   3.0   4.0   4.0   3.0   NaN       587.65  10884.00  7181.20   
..        ...   ...   ...   ...   ...   ...          ...       ...      ...   
867       4.0   2.0   4.0   3.0   3.0   NaN     10391.50       NaN  6767.25   
868       0.0   0.0   0.0   0.0   0.0   NaN      6313.00   7894.67  1276.00   
869       3.0   4.0   2.0   6.0   3.0   NaN      2222.05   5598.86  4053.78   
870       2.0   2.0   2.0   2.0   6.0   NaN      2340.12   3405.91  5957.57   
871       3.0   2.0   2.0   0.0   3.0   NaN      4048.12   1455.67  2577.81   

0                                                                         \
file_id    LGA04    LGA05    LGA06     LGA07    LGA08     LGA09    LGA10   
0         922.20   602.54   172.05    398.12   431.00    490.03   411.97   
1        3897.86  3497.73      NaN   5537.50   879.98   2780.87   885.48   
2        1792.04  1105.63   236.69   1737.04  1125.13    744.50   668.69   
3         509.23  1099.32      NaN    599.62   402.63    296.28   252.77   
4         506.65  3649.75      NaN    463.26   497.42   1513.62   285.57   
..           ...      ...      ...       ...      ...       ...      ...   
867      1672.38  1826.00  1678.64   1106.18   815.06    496.72   440.10   
868      5689.20  8805.00  4638.00  13446.50  5519.00  11128.00      NaN   
869      2977.27  3496.42  1751.73   1940.00  1203.58   1359.39   594.79   
870      1554.07  1708.48  1327.21    904.33  2122.85   1580.95  1064.82   
871       470.48   717.26   672.58    673.94   529.17   1050.97   951.33   

0                                                meanNumRewards              \
file_id    LGA11    LGA12   LGA13    LGA14 LGA15          LGA01 LGA02 LGA03   
0         393.97   322.64  323.17   359.99   NaN           2.18  2.00  2.19   
1         448.39  2447.00  438.33   501.59   NaN           2.33  2.00  2.33   
2         449.37   479.85  490.76   517.37   NaN           2.00  2.00  2.10   
3         327.90   327.15  327.00   470.84   NaN           2.00  2.08  2.10   
4        2023.90   428.62  372.03   385.48   NaN           3.38  0.00  2.00   
..           ...      ...     ...      ...   ...            ...   ...   ...   
867      1198.58   504.84  424.74   477.81   NaN           0.00  0.00  0.00   
868      8579.00      NaN     NaN  7747.00   NaN           0.00  0.00  0.00   
869       453.38   757.00  334.78  1013.27   NaN           2.00  0.00  2.00   
870      1387.14  1759.83  976.57   233.72   NaN           2.00  2.00  2.00   
871       923.38   910.02  831.06   926.78   N

In [540]:
out.head()

0       Subject maxBurst                                                  \
file_id            LGA01 LGA02 LGA03 LGA04 LGA05 LGA06 LGA07 LGA08 LGA09   
0          F401      3.0   2.0   3.0   3.0   2.0   3.0   2.0   4.0   5.0   
1          F402      3.0   2.0   3.0   0.0   2.0   NaN   2.0   3.0   4.0   
2          F403      2.0   2.0   3.0   2.0   2.0   4.0   2.0   2.0   2.0   
3          F404      2.0   3.0   3.0   4.0   3.0   NaN   4.0   4.0   4.0   
4          F405      6.0   0.0   2.0   5.0   3.0   NaN   4.0   3.0   4.0   

0                                           meanInterval                     \
file_id LGA10 LGA11 LGA12 LGA13 LGA14 LGA15        LGA01     LGA02    LGA03   
0         3.0   4.0   4.0   5.0   3.0   NaN       652.64    618.26   505.85   
1         3.0   4.0   2.0   4.0   6.0   NaN      1011.34   3409.00   969.55   
2         2.0   2.0   3.0   3.0   3.0   NaN      1945.30   1483.00   686.46   
3         5.0   5.0   4.0   5.0   4.0   NaN       988.86    624.13   834.39   
4         4.0   3.0   4.0   4.0   3.0   NaN       587.65  10884.00  7181.20   

0                                                                              \
file_id    LGA04    LGA05   LGA06    LGA07    LGA08    LGA09   LGA10    LGA11   
0         922.20   602.54  172.05   398.12   431.00   490.03  411.97   393.97   
1        3897.86  3497.73     NaN  5537.50   879.98  2780.87  885.48   448.39   
2        1792.04  1105.63  236.69  1737.04  1125.13   744.50  668.69   449.37   
3         509.23  1099.32     NaN   599.62   402.63   296.28  252.77   327.90   
4         506.65  3649.75     NaN   463.26   497.42  1513.62  285.57  2023.90   

0                                      meanNumRewards                          \
file_id    LGA12   LGA13   LGA14 LGA15          LGA01 LGA02 LGA03 LGA04 LGA05   
0         322.64  323.17  359.99   NaN           2.18  2.00  2.19  2.15  2.00   
1        2447.00  438.33  501.59   NaN           2.33  2.00  2.33  0.00  2.00   
2         479.85  490.76  517.37   NaN           2.00  2.00  2.10  2.00  2.00   
3         327.15  327.00  470.84   NaN           2.00  2.08  2.10  2.56  2.12   
4         428.62  372.03  385.48   NaN           3.38  0.00  2.00  3.15  2.50   

0                                                                    \
file_id LGA06 LGA07 LGA08 LGA09 LGA10 LGA11 LGA12 LGA13 LGA14 LGA15   
0        2.04  2.00  2.16  2.40  2.10  2.29  2.33  2.23  2.18   NaN   
1         NaN  2.00  2.12  3.50  2.14  2.17  2.00  2.47  2.88   NaN   
2        2.33  2.00  2.00  2.00  2.00  2.00  2.11  2.10  2.07   NaN   
3         NaN  2.14  2.45  2.59  2.66  2.68  2.56  2.76  2.53   NaN   
4         NaN  3.00  2.12  2.29  2.56  2.20  2.44  2.28  2.10   NaN   

0       modeInterval                                                           \
file_id        LGA01    LGA02    LGA03   LGA04   LGA05  LGA06    LGA07  LGA08   
0               23.0     43.0     24.0    29.0    22.0  127.0     58.0   74.0   
1               29.0   4056.0     26.0  5636.0  3572.0    NaN  12347.0  332.0   
2             2488.0     35.0     31.0    51.0    48.0  291.0  10737.0  775.0   
3             6055.0     25.0    192.0    64.0  4342.0    NaN     97.0   23.0   
4               21.0  15989.0  29173.0    20.0   131.0    NaN     24.0   22.0   

0                                                    numBurst              \
file_id   LGA09  LGA10 LGA11 LGA12 LGA13 LGA14 LGA15    LGA01 LGA02 LGA03   
0          24.0  557.0  23.0  22.0  24.0  33.0   NaN     17.0  12.0  16.0   
1          25.0   24.0  27.0  48.0  22.0  27.0   NaN      6.0   2.0   9.0   
2        1040.0   68.0  27.0  33.0  47.0  35.0   NaN      4.0   3.0  10.0   
3          26.0   26.0  22.0  32.0  22.0  30.0   NaN      8.0  12.0  10.0   
4          23.0   39.0  36.0  24.0  24.0  22.0   NaN     13.0   0.0   2.0   

0                                                                          \
file_id LGA04 LGA05 LGA06 LGA07 LGA08 LGA09 LGA10 LGA11 LGA12 LGA13 LGA14   
0        13.0  19

In [541]:
out.to_csv('calculations_LGA01-14.csv')

In [580]:
rows[178]

0       Subject maxBurst meanInterval meanNumRewards modeInterval numBurst  \
file_id            LGA01        LGA01          LGA01        LGA01    LGA01   
0          M552      2.0      2170.07            2.0         26.0      4.0   

0       pctRewards stdInterval maxBurst meanInterval meanNumRewards  \
file_id      LGA01       LGA01    LGA02        LGA02          LGA02   
0             53.0     3329.56      0.0      3927.43            0.0   

0       modeInterval numBurst pctRewards stdInterval maxBurst meanInterval  \
file_id        LGA02    LGA02      LGA02       LGA02    LGA03        LGA03   
0              533.0      0.0        0.0     3048.81      0.0      7927.67   

0       meanNumRewards modeInterval numBurst pctRewards stdInterval maxBurst  \
file_id          LGA03        LGA03    LGA03      LGA03       LGA03    LGA04   
0                  0.0       9197.0      0.0        0.0     5333.49      2.0   

0       meanInterval meanNumRewards modeInterval numBurst pctRewards  \
file_id        LGA04          LGA04        LGA04    LGA04      LGA04   
0             6220.8            2.0        985.0      1.0       33.0   

0       stdInterval maxBurst meanInterval meanNumRewards modeInterval  \
file_id       LGA04    LGA05        LGA05          LGA05        LGA05   
0           8347.51      0.0       7123.5            0.0       6825.0   

0       numBurst pctRewards stdInterval maxBurst meanInterval meanNumRewards  \
file_id    LGA05      LGA05       LGA05    LGA06        LGA06          LGA06   
0            0.0        0.0     3102.73      2.0      4115.56            2.0   

0       modeInterval numBurst pctRewards stdInterval maxBurst meanInterval  \
file_id        LGA06    LGA06      LGA06       LGA06    LGA07        LGA07   
0              235.0      1.0       20.0     4038.04      0.0       6187.5   

0       meanNumRewards modeInterval numBurst pctRewards stdInterval maxBurst  \
file_id          LGA07        LGA07    LGA07      LGA07       LGA07    LGA08   
0                  0.0        134.0      0.0        0.0     11151.6      0.0   

0       meanInterval meanNumRewards modeInterval numBurst pctRewards  \
file_id        LGA08          LGA08        LGA08    LGA08      LGA08   
0             2466.0            0.0       2466.0      0.0        0.0   

0       stdInterval maxBurst meanInterval meanNumRewards modeInterval  \
file_id       LGA08    LGA09        LGA09          LGA09        LGA09   
0               0.0      2.0      4824.17            2.0         39.0   

0       numBurst pctRewards stdInterval maxBurst meanInterval meanNumRewards  \
file_id    LGA09      LGA09       LGA09    LGA10        LGA10          LGA10   
0            1.0       29.0     6063.87      2.0      5200.75            2.0   

0       modeInterval numBurst pctRewards stdInterval maxBurst        \
file_id        LGA10    LGA10      LGA10       LGA10    LGA11 LGA11   
0               89.0      1.0       40.0     7175.05      0.0   0.0   

0       meanInterval          meanNumRewards       modeInterval          \
file_id        LGA11    LGA11          LGA11 LGA11        LGA11   LGA11   
0           10742.67  10165.5            0.0   0.0       3600.0  4520.0   

0       numBurst       pctRewards       stdInterval         maxBurst  \
file_id    LGA11 LGA11      LGA11 LGA11       LGA11   LGA11    LGA13   
0            0.0   0.0        0.0   0.0     5110.97  5645.5      0.0   

0       meanInterval meanNumRewards modeInterval numBurst pctRewards  \
file_id        LGA13          LGA13        LGA13    LGA13      LGA13   
0             7736.5            0.0       1805.0      0.0        0.0   

0       stdInterval maxBurst meanInterval meanNumRewards modeInterval  \
file_id       LGA13    LGA14        LGA14          LGA14        LGA14   
0            5931.5      2.0         95.0            2.0         95.0   

0       numBurst pctRewards stdInterval  
file_id    LGA14      LGA14       LGA14  
0            1.0      100.0         0.0

In [583]:
out[out['Subject']=='M755']

0       Subject maxBurst                                                  \
file_id            LGA01 LGA02 LGA03 LGA04 LGA05 LGA06 LGA07 LGA08 LGA09   
320        M755      NaN   0.0   0.0   0.0   0.0   0.0   0.0   0.0   NaN   

0                                           meanInterval                    \
file_id LGA10 LGA11 LGA12 LGA13 LGA14 LGA15        LGA01 LGA02 LGA03 LGA04   
320       2.0   0.0   3.0   2.0   2.0   2.0          NaN   NaN   NaN   NaN   

0                                                                         \
file_id LGA05 LGA06 LGA07 LGA08 LGA09    LGA10    LGA11    LGA12   LGA13   
320       NaN   NaN   NaN   NaN   NaN  1732.73  2840.18  3031.09  960.87   

0                         meanNumRewards                                      \
file_id    LGA14    LGA15          LGA01 LGA02 LGA03 LGA04 LGA05 LGA06 LGA07   
320      1117.32  1010.54            NaN   0.0   0.0   0.0   0.0   0.0   0.0   

0                                                       modeInterval        \
file_id LGA08 LGA09 LGA10 LGA11 LGA12 LGA13 LGA14 LGA15        LGA01 LGA02   
320       0.0   NaN   2.0   0.0   2.5   2.0   2.0   2.0          NaN   NaN   

0                                                                            \
file_id LGA03 LGA04 LGA05 LGA06 LGA07 LGA08 LGA09 LGA10   LGA11 LGA12 LGA13   
320       NaN   NaN   NaN   NaN   NaN   NaN   NaN  88.0  3708.0  68.0  28.0   

0                     numBurst                                            \
file_id  LGA14  LGA15    LGA01 LGA02 LGA03 LGA04 LGA05 LGA06 LGA07 LGA08   
320      724.0  385.0      NaN   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

0                                                 pctRewards              \
file_id LGA09 LGA10 LGA11 LGA12 LGA13 LGA14 LGA15      LGA01 LGA02 LGA03   
320       NaN   5.0   0.0   2.0   1.0   5.0   3.0        NaN   NaN   0.0   

0                                                                          \
file_id LGA04 LGA05 LGA06 LGA07 LGA08 LGA09 LGA10 LGA11 LGA12 LGA13 LGA14   
320       NaN   NaN   NaN   NaN   NaN   NaN  43.0   0.0  42.0   6.0  34.0   

0             stdInterval                                                  \
file_id LGA15       LGA01 LGA02 LGA03 LGA04 LGA05 LGA06 LGA07 LGA08 LGA09   
320      14.0         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

0                                                           
file_id   LGA10    LGA11    LGA12   LGA13   LGA14    LGA15  
320      2316.3  2344.73  3545.63  443.35  824.24  1280.26